#### **12. Implement batch gradient descent with early stopping for softmax regression without using Scikit-Learn, only NumPy. Use it on a classification task such as the iris dataset.**

In [13]:
import numpy as np

In [1]:
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)

In [4]:
X = iris.data.values
X.shape # 150 instances with 4 features each

(150, 4)

In [8]:
y = iris.target_names[iris.target]
print(f"y has {y.shape[0]} instances") 

y has 150 instances
<class 'numpy.ndarray'>


In [9]:
# Splitting the data 
from sklearn.model_selection import train_test_split

X_train, y_train, X_test, y_test = train_test_split(X, y, random_state=42)

In [11]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print('*'*50)
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (112, 4)
y_train shape: (38, 4)
**************************************************
X_test shape: (112,)
y_test shape: (38,)


In [17]:
# Implementing batch gradient descent
learning_rate = 0.02
num_epochs = 1000
num_instances = X_train.shape[0]

np.random.seed(42)
# Randomly initialize weights for each feature
theta = np.random.randn(X_train.shape[1], 1)
print(f"Initial Theta: {theta}")

for epoch in range(num_epochs):
    gradients = 2 / num_instances * X_train.T @ (X_train @ theta - y_train)
    print(gradients)

Initial Theta: [[ 0.49671415]
 [-0.1382643 ]
 [ 0.64768854]
 [ 1.52302986]]


ValueError: operands could not be broadcast together with shapes (112,1) (38,4) 